# Machine Learning Engineer Nanodegree

## Capstone Project - Code Base

Mathias von Kaiz  
August 1st, 2017

**This code base is to evaluate functions, algorithms for developing the final Robot class.**

In [26]:
from maze import Maze
#from robot import Robot
import sys
import numpy as np
import random
import copy

from IPython.core import debugger
debug = debugger.Pdb().set_trace

In [72]:
_maze = 'maze1'
_timespan = 100
_maxtimespan = 1000

In [73]:
dir_mazes = {'maze1': 'test_maze_01.txt',
            'maze2': 'test_maze_02.txt',
            'maze3': 'test_maze_03.txt'}
testmaze = Maze( str(dir_mazes[_maze]) )


#####################

_init = [0, 0]
_distance = [0, 1, 2, 3]

#####################
_steering = ['left', 'front', 'right']
_dir_steering_to_rotation = {'left': -90, 'front': 0, 'right': 90}
_dir_steering_to_sensor_index = {'left': 0, 'front': 1, 'right': 2}

_dir_map_value = {'N': 1, 'E': 2, 'S': 4 , 'W': 8 }

_heading = ['u', 'r', 'd', 'l']
_dir_heading_to_map = {'u': 'N', 'r': 'E', 'd': 'S', 'l': 'W',
                       'up': 'N', 'right': 'E', 'down': 'S', 'left': 'W'}
_dir_int = {'u': 1, 'r': 2, 'd': 4, 'l': 8,
            'up': 1, 'right': 2, 'down': 4, 'left': 8}

_dir_heading_to_symbol = {'u': '^', 'r': '>', 'd': 'v', 'l': '<',
                          'up': '^', 'right': '>', 'down': 'v', 'left': '<'}


_dir_sensors = {'u': ['l', 'u', 'r'], 'r': ['u', 'r', 'd'],
               'd': ['r', 'd', 'l'], 'l': ['d', 'l', 'u'],
               'up': ['l', 'u', 'r'], 'right': ['u', 'r', 'd'],
               'down': ['r', 'd', 'l'], 'left': ['d', 'l', 'u']}
_dir_move = {'u': [0, 1], 'r': [1, 0], 'd': [0, -1], 'l': [-1, 0],
            'up': [0, 1], 'right': [1, 0], 'down': [0, -1], 'left': [-1, 0]}
_dir_steering_cost = {'u': 20, 'r': 20, 'd': 20, 'l': 20,
            'up': 20, 'right': 20, 'down': 20, 'left': 20}
#_dir_steering_cost_dir_cost = {'u': 1, 'r': 1, 'd': 1, 'l': 1,
#            'up': 1, 'right': 1, 'down': 1, 'left': 1}
_dir_reverse = {'u': 'd', 'r': 'l', 'd': 'u', 'l': 'r',
               'up': 'd', 'right': 'l', 'down': 'u', 'left': 'r'}





In [74]:
class Map(object):
    def __init__(self, map_dim):
        '''
        Use the initialization function to set up attributes
        '''

        self.map_dim = map_dim
        self.grid = [[-1 for col in range(map_dim)] for row in range(map_dim)]
        self.visited = [[-1 for col in range(map_dim)] for row in range(map_dim)]
        self.moved = [[' ' for col in range(map_dim)] for row in range(map_dim)]
        
        
        
    def reset(self):
        '''
        Resets properties that are used in following runs
        '''
        
        self.visited = [[-1 for col in range(self.map_dim)] for row in range(self.map_dim)]
        self.moved = [[' ' for col in range(self.map_dim)] for row in range(self.map_dim)]
        
        
    def is_permissible(self, cell, direction):
        """
        Returns a boolean designating whether or not a cell is passable in the
        given direction. Cell is input as a list. Directions may be
        input as single letter 'u', 'r', 'd', 'l', or complete words 'up', 
        'right', 'down', 'left'.
        """
        
        if self.is_unknown(cell):
            return False
        
        try:
            return (self.grid[cell[0]][cell[1]] & _dir_int[direction] != 0)
        except:
            print cell, direction, ' is a wall!'
           
        
        
    def is_unknown(self, cell):
        '''
        Checks if a given cell is unknown (not yet explored/visited)
        '''
        
        return self.grid[cell[0]][cell[1]] == -1
        
        
        
    def expand(self, location, heading, sensors):
        '''
        Expand function that documents explored cells
        '''
        
        value = 0
        # TODO refactor
        # check sensor steering
        for direction in _steering:
            if sensors[_dir_steering_to_sensor_index[direction]] > 0:
                i = _dir_map_value[_dir_heading_to_map[_dir_sensors[heading][_dir_steering_to_sensor_index[direction]]]]
                value += i
                
        # map reverse if not at starting position
        # TODO: refactor fo better way
        if (location != _init):
            value += _dir_map_value[_dir_heading_to_map[_dir_reverse[heading]]]
        
        # set value
        if self.grid[location[0]][location[1]] == -1:
            self.grid[location[0]][location[1]] = value
            
            
    def check_coverage(self):
        '''
        Calcluates the exploration coverage of the map
        '''
        unique, counts = np.unique(self.visited, return_counts=True)
        dir_coverage = dict(zip(unique, counts))
        total = np.prod(np.array(self.visited).shape)
        
        if float(dir_coverage[1]) / float(total) >= 0.7:
            return True
        else:
            return False 
                

In [75]:
class Plan(object):
    
    def __init__(self, robot, cost = 1):
        '''
        Use the initialization function to set up attributes
        '''
        
        self.robot = robot
        self.map = robot.map
        self.grid_size = len(robot.map.grid[0])
        
        self.start = robot.get_start()
        self.init = robot.location
        self.set_goal()
        
        self.is_exploring = True
        self.is_goal_reached = False
        self.is_looking_for_start = False
        
        self.cost = cost
        self.make_heuristic()
        
        self.path = []
        self.policy = []
        self.expand = []
       
    
    def reset(self):
        '''
        Resets properties that are used in following runs
        '''
        
        self.start = self.robot.get_start()
        self.init = self.robot.location
        self.set_goal()
        
        self.is_exploring = False
        self.is_goal_reached = False
        self.is_looking_for_start = False
        
        self.path = []
        self.policy = []
        self.expand = []
        
     
    def set_goal(self, goal_type='Goal'):
        '''
        Calculate goal function for a grid
        '''
        
        if goal_type == 'Goal':
            self.goal = [self.grid_size/2 - 1, self.grid_size/2]
        elif goal_type == 'Start':
            self.goal = self.robot.start
            
    
    def set_goal_reached(self):
        '''
        Calculated for goal reach
        '''
        
        self.is_goal_reached = self.robot.location[0] in self.goal and self.robot.location[1] in self.goal
    
    
    def set_steering_by_path(self, idx = 0):
        '''
        Returns steering angle for given heading
        '''
        
        steering = _steering[1]
        
        if len(self.path) > 0:
            try:
                for direction in _steering:
                    if _dir_sensors[self.robot.heading][_dir_steering_to_sensor_index[direction]] == self.path[idx]:
                        steering = direction
            except IndexError:
                print('--- No path segmet found ---')     
        else:
            print('--- No path found ---')
                    
        return steering
    
    
    def check_movement(self, path_step, heading):
        '''
        Checks if we need to do a reverse movement
        '''
        if _dir_reverse[heading] == path_step:
            return -1
        else:
            return 1
    
    
    def make_heuristic(self):
        '''
        Make heuristic function for a grid
        '''
        
        self.heuristic = [[0 for row in range(self.grid_size)] 
                          for col in range(self.grid_size)]
        
        for i in range(self.grid_size):    
            for j in range(self.grid_size):
                self.heuristic[i][j] = abs(i - self.goal[0]) + abs(j - self.goal[1])
                    
                    
    def random(self):
        '''
        Random search for goal
        '''
        
        steering = random.choice(_steering) 
        movement = random.choice(_distance)
        
        return steering, movement
    

    #     def recursive_search(self, init):
#         '''
#         Recursive search for goal
#         '''
        
#         # Init arrays
#         closed = [[0 for col in range(self.grid_size)] for row in range(self.grid_size)]
#         closed[init[0]][init[1]] = 1
#         action = [[-1 for col in range(self.grid_size)] for row in range(self.grid_size)]

#         # Init parameters
#         x = init[0]
#         y = init[1]
        
#         found = False
#         resign = False
#         count = 0
        
        # Set open on beginning of algorithm
#         open = [[x, y]]
        
#         # Loop while goal is not found or stuck
#         while not found and not resign:
            
#             # Goal found
#             if len(open) == 0:
#                 resign = True
#                 print ("Failed as not path found")
            
#             # Goal not yet found
#             else:
#                 #open.sort()
#                 #open.reverse()
#                 #next = open.pop()
#                 #x = next[0]
#                 #y = next[1]
                
#                 # check if we are done
#                 if x == self.goal[0] and y == self.goal[1]:
#                     found = True

#                 else:
#                     for i in range(len(_heading)):
#                         heading = _dir_move[_heading[i]]

#                         x2 = x
#                         y2 = y

#                         # check if we can proceed or if unknown
#                         if self.map.is_permissible([x2, y2], _heading[i]) or self.map.is_unknown([x2, y2]):
#                             x2 = x + heading[0]
#                             y2 = y + heading[1]
                            
#                             if  x2 >= 0 and x2 < self.grid_size and y2 >= 0 and y2 < self.grid_size:
#                                 if closed[x2][y2] == 0:
#                                     closed[x2][y2] = 1
#                                     action[x2][y2] = i
        
        
        
#         # Set optimal policy
#         policy = [[' ' for col in range(self.grid_size)] for row in range(self.grid_size)]
#         x = self.goal[0]
#         y = self.goal[1]
#         policy[x][y] = '*'
        
#         # Set optimal path
#         invpath = []
#         while x != init[0] or y != init[1]:
#             x2 = x - _dir_move[_heading[action[x][y]]][0]
#             y2 = y - _dir_move[_heading[action[x][y]]][1]
#             policy[x2][y2] = _dir_heading_to_symbol[_heading[action[x][y]]]
#             invpath.append(_heading[action[x][y]])
#             x = x2
#             y = y2
            
#         path = []
#         for i in range(len(invpath)):
#             path.append(invpath[len(invpath) - 1 - i])
            
#         # Return path, policy and expand arrays
#         return path, policy
    
    
#     def recursive(self, path=False):
#         '''
#         Recursive main
#         '''
        
#         # perform recursive search
#         self.path, self.policy = self.recursive_search(self.init)
        
#         # set steering
#         if len(self.path) != 0:   
#             steering = self.set_steering_by_path()
        
#         # differentiate between path searching or next steering
#         if path:
#             # return optimal path and policy
#             return self.path, self.policy
#         else:
#             # return steering direction and movement 1
#             return steering, 1



    def recursive_search(self, init):
        '''
        Recursive search for goal
        '''
        
#        closed = [[0 for col in range(self.grid_size)] for row in range(self.grid_size)]
        action = [[-1 for col in range(self.grid_size)] for row in range(self.grid_size)]
        
        # initialize policy or assign to plan policy to keep track
        if len(self.policy) == 0:
            policy = [[' ' for col in range(self.grid_size)] for row in range(self.grid_size)]
            policy[self.goal[0]][self.goal[1]] = 'x'
        else:
            policy = self.policy
            
        # initialize path or assign to plan path to keep track
        if len(self.path) == 0:
            path = []
        else:
            path = self.path
            
        
            
#         try:
#             policy
#         except NameError as e:
#             policy = [[' ' for col in range(self.grid_size)] for row in range(self.grid_size)]
#             policy[self.goal[0]][self.goal[1]] = '*'
#         else:
#             policy = self.policy
            
#         try:
#             path
#         except NameError as e:
#             path = []
#         else:
#             path = self.path
        
        
        goal = False
        found = True
        next_move = False
        
        
        x = init[0]
        y = init[1]
        
        while not goal and not next_move:
            found_heading = []
            found_resign = False
            
            # check if we are done
            if x == self.goal[0] and y == self.goal[1]:
                goal = True

            for i in range(len(_heading)):
                heading = _dir_move[_heading[i]]

                x2 = x
                y2 = y
                
                if  x2 >= 0 and x2 < self.grid_size and y2 >= 0 and y2 < self.grid_size:
                    
                    # check if we can proceed
                    if self.map.is_permissible([x2, y2], _heading[i]):
                        x2 = x + heading[0]
                        y2 = y + heading[1]
                        
                        if  x2 >= 0 and x2 < self.grid_size and y2 >= 0 and y2 < self.grid_size and self.map.is_unknown([x2, y2]):
                            action[x][y] = i
                            policy[x][y] = _dir_heading_to_symbol[_heading[i]]
                            path.append(_heading[action[x][y]])
                            found_heading.append(True)
                            next_move = True
                            break
                            
                        else:
                            found_heading.append(False)
                    else:
                        found_heading.append(False)
                else:
                    found_heading.append(False)
                            
            # iterate through all found headings nad check for possible path
            for i in range(len(found_heading)):
                if found_heading[i]:
                    found_resign = True
                    
            found = found_resign
            next_move = True
                            
        # if we have not found a path, go back one step
        if not found:
            pass
            # remove last coming from

            
        # Set optimal policy
#         policy = [[' ' for col in range(self.grid_size)] for row in range(self.grid_size)]
#         x = self.robot.location[0]
#         y = self.robot.location[1]
#         policy[x][y] = '*'
        
#         # Set optimal path
#         invpath = []
#         debug()
#         # check if we are on start location
#         if x == self.robot.start[0] and y == self.robot.start[1]:
#             x2 = x + _dir_move[_heading[action[x][y]]][0]
#             y2 = y + _dir_move[_heading[action[x][y]]][1]
#             policy[x2][y2] = _dir_heading_to_symbol[_heading[action[x][y]]]
#             invpath.append(_heading[action[x][y]])
#             x = x2
#             y = y2
#         else:
#             while x != self.robot.init[0] or y != self.robot.init[1]:
#                 x2 = x - _dir_move[_heading[action[x][y]]][0]
#                 y2 = y - _dir_move[_heading[action[x][y]]][1]
#                 policy[x2][y2] = _dir_heading_to_symbol[_heading[action[x][y]]]
#                 invpath.append(_heading[action[x][y]])
#                 x = x2
#                 y = y2
            
#         path = []
#         for i in range(len(invpath)):
#             path.append(invpath[len(invpath) - 1 - i])
            
            #             print("Need to go back one step")
#             action[x][y] = 99
#             policy[x][y] = _dir_heading_to_symbol[_heading[i]]
            # iterate again through the headings and check where we coming from
#             for i in range(len(_heading)):
#                 heading = _dir_move[_heading[i]]
#                 x2 = x
#                 y2 = y
#                 # check if we can proceed
#                 if self.map.is_permissible([x2, y2], _heading[i]):
#                     action[x][y] = i
#                     policy[x][y] = _dir_heading_to_symbol[_heading[i]]
#                     break
        
     
        # Set optimal policy
#         x = self.init[0]
#         y = self.init[1]
        
#         # Set optimal path
#         path = []
#         while (x != self.goal[0] or y != self.goal[1]) and action[x][y] > -1:
#             x2 = x + _dir_move[_heading[action[x][y]]][0]
#             y2 = y + _dir_move[_heading[action[x][y]]][1]
#             path.append(_heading[action[x][y]])
#             x = x2
#             y = y2
            
            
        # Return path, policy arrays
        return path, policy 
            
        
        

    def recursive(self, path=False):
        '''
        Recursive main
        '''
        
        # assign class variables
        self.path, self.policy = self.recursive_search(self.init)
        
        #debug()
        
        # set steering
        if len(self.path) != 0:
            # as we do not have a path in future but in past
            # we need to past the last path element
            idx = len(self.path) - 1
            steering = self.set_steering_by_path(idx)
        
        # differentiate between path searching or next steering
        if path:
            # return optimal path and policy
            return self.path, self.policy
        else:
            # check for reverse movement
            movement = self.check_movement(self.path[0], self.robot.heading)
            # return steering direction and movement
            return steering, movement
        
        
        
    
    def astar_search(self, init, is_path):
        '''
        A* search for goal
        '''
        
        # Init arrays
        closed = [[0 for col in range(self.grid_size)] for row in range(self.grid_size)]
        closed[init[0]][init[1]] = 1
        expand = [[-1 for col in range(self.grid_size)] for row in range(self.grid_size)]
        action = [[-1 for col in range(self.grid_size)] for row in range(self.grid_size)]

        # Init parameters
        x = init[0]
        y = init[1]
        g = 0
        h = self.heuristic[x][y]
        f = g + h

        found = False
        resign = False
        count = 0
        
        # Set open on beginning of algorithm
        open = [[f, h, g, x, y]]
        
        # Loop while goal is not found or stuck
        while not found and not resign:
            
            # Goal found
            if len(open) == 0:
                resign = True
                print ("Failed as not path found")
            
            # Goal not yet found
            else:
                open.sort()
                open.reverse()
                next = open.pop()
                x = next[3]
                y = next[4]
                g = next[2]

                expand[x][y] = count
                count += 1

                # check if we are done
                if x == self.goal[0] and y == self.goal[1]:
                    found = True

                else:
                    for i in range(len(_heading)):
                        heading = _dir_move[_heading[i]]
                        
                        x2 = x
                        y2 = y

                        # check if we can proceed or if unknown
                        if is_path:
                            if self.map.is_permissible([x2, y2], _heading[i]):
                                x2 = x + heading[0]
                                y2 = y + heading[1]

                                if  x2 >= 0 and x2 < self.grid_size and y2 >= 0 and y2 < self.grid_size:
                                    if closed[x2][y2] == 0:
                                        # Steering is accounted with higher costs
                                        # to support direct movements
                                        if _dir_move[self.robot.heading] == heading:
                                            g2 = g + self.cost
                                        else:
                                            g2 = g + _dir_steering_cost[_heading[i]]

                                        h2 = self.heuristic[x2][y2]
                                        f2 = g2 + h2
                                        open.append([f2, h2, g2, x2, y2])
                                        closed[x2][y2] = 1
                                        action[x2][y2] = i
                        else:
                            if self.map.is_permissible([x2, y2], _heading[i]) or self.map.is_unknown([x2, y2]):
                                x2 = x + heading[0]
                                y2 = y + heading[1]

                                if  x2 >= 0 and x2 < self.grid_size and y2 >= 0 and y2 < self.grid_size:
                                    if closed[x2][y2] == 0:
                                        # Steering is accounted with higher costs
                                        # to support direct movements
                                        if self.robot.heading == heading:
                                            g2 = g + self.cost
                                        else:
                                            g2 = g + _dir_steering_cost[_heading[i]]

                                        h2 = self.heuristic[x2][y2]
                                        f2 = g2 + h2
                                        open.append([f2, h2, g2, x2, y2])
                                        closed[x2][y2] = 1
                                        action[x2][y2] = i
        
        
        # Set optimal policy
        policy = [[' ' for col in range(self.grid_size)] for row in range(self.grid_size)]
        x = self.goal[0]
        y = self.goal[1]
        policy[x][y] = 'x'
        
        # Set optimal path
        invpath = []
        
        while x != init[0] or y != init[1]:
            x2 = x - _dir_move[_heading[action[x][y]]][0]
            y2 = y - _dir_move[_heading[action[x][y]]][1]
            policy[x2][y2] = _dir_heading_to_symbol[_heading[action[x][y]]]
            invpath.append(_heading[action[x][y]])
            x = x2
            y = y2
            
        path = []
        for i in range(len(invpath)):
            path.append(invpath[len(invpath) - 1 - i])
            
        # Return path, policy and expand arrays
        return path, policy, expand
    
    
    def astar(self, path=False):
        '''
        A* main
        '''
                
        if self.heuristic == []:
            raise ValueError, "Heuristic must be defined to run A*"
        
        # perform a* search
        self.path, self.policy, self.expand = self.astar_search(self.init, path)
        
        # set steering
        if len(self.path) != 0:   
            steering = self.set_steering_by_path()
        
        # differentiate between path searching or next steering
        if path:
            # return optimal path and policy
            return self.path, self.policy
        else:
            # check for reverse movement
            movement = self.check_movement(self.path[0], self.robot.heading)
            
            # return steering direction and movement
            return steering, movement
    
    
    def dp_search(self, init):
        '''
        Dynamic Programming search for goal
        '''
        
        value = [[999 for col in range(self.grid_size)] for col in range(self.grid_size)]
        action = [[-1 for col in range(self.grid_size)] for row in range(self.grid_size)]
        
        policy = [[' ' for col in range(self.grid_size)] for row in range(self.grid_size)]
        policy[self.goal[0]][self.goal[1]] = '*'
    
        change = True
        while change:
            change = False
            
            for x in range(self.grid_size):
                for y in range(self.grid_size):

                    if x == self.goal[0] and y == self.goal[1]:
                        if value[x][y] > 0:
                            value[x][y] = 0
                            change = True

                    else:
                        for i in range(len(_heading)):
                            heading = _dir_move[_heading[i]]

                            x2 = x
                            y2 = y

                            # check if we can proceed 
                            if self.map.is_permissible([x2, y2], _heading[i]):
                                x2 = x + heading[0]
                                y2 = y + heading[1]

                                if  x2 >= 0 and x2 < self.grid_size and y2 >= 0 and y2 < self.grid_size:
                                    v2 = value[x2][y2] + self.cost
                                    
                                    if v2 < value[x][y]:
                                        change = True
                                        value[x][y] = v2
                                        action[x][y] = i
                                        policy[x][y] = _dir_heading_to_symbol[_heading[i]]

        # Set optimal policy
        x = init[0]
        y = init[1]
        
        # Set optimal path
        path = []
        while x != self.goal[0] or y != self.goal[1]:
            x2 = x + _dir_move[_heading[action[x][y]]][0]
            y2 = y + _dir_move[_heading[action[x][y]]][1]
            path.append(_heading[action[x][y]])
            x = x2
            y = y2
            
        return path, policy
    

    def dp(self, path=False):
        '''
        Dynamic Programming main
        '''
        
        # perform dynamic programming search
        self.path, self.policy = self.dp_search(self.init)
        
        # set steering
        if len(self.path) != 0:   
            steering = self.set_steering_by_path()
        
        # differentiate between path searching or next steering
        if path:
            # return optimal path and policy
            return self.path, self.policy
        else:
            movement = self.check_movement(self.path[0], self.robot.heading)
            # check for reverse movement
            
            # return steering direction and movement
            return steering, movement

In [76]:
class Robot(object):
    def __init__(self, map_dim):
        '''
        Use the initialization function to set up attributes that your robot
        will use to learn and navigate the maze. Some initial attributes are
        provided based on common information, including the size of the maze
        the robot is placed in.
        '''
        self.start = [0, 0]
        self.location = self.get_start()
        self.heading = 'up'
        
        self.map = Map(map_dim)
        self.plan = Plan(self)
        
        self.timespan = _timespan
        self.timestep = 0
        self.explored_timestep = 0
        self.overall_tiestep = 0
        self.max_timestep = _maxtimespan
        
        
    def reset(self):
        '''
        Resets properties that are used in following runs
        '''
        
        self.location = self.get_start()
        self.heading = 'up'
        self.explored_timestep = self.timestep
        self.timestep = 0
            
    
    def set_heading(self, steering):
        '''
        Sets heading based on given steering
        '''
        
        self.heading = _dir_sensors[self.heading][_dir_steering_to_sensor_index[steering]]
        
        
    def get_start(self):
        '''
        Getter for start cell
        '''
        return copy.copy(self.start)
    
    

    def next_move(self, sensors):
        '''
        Use this function to determine the next move the robot should make,
        based on the input from the sensors after its previous move. Sensor
        inputs are a list of three distances from the robot's left, front, and
        right-facing sensors, in that order.

        Outputs should be a tuple of two values. The first value indicates
        robot rotation (if any), as a number: 0 for no rotation, +90 for a
        90-degree rotation clockwise, and -90 for a 90-degree rotation
        counterclockwise. Other values will result in no rotation. The second
        value indicates robot movement, and the robot will attempt to move the
        number of indicated squares: a positive number indicates forwards
        movement, while a negative number indicates backwards movement. The
        robot may move a maximum of three units per turn. Any excess movement
        is ignored.

        If the robot wants to end a run (e.g. during the first training run in
        the maze) then returing the tuple ('Reset', 'Reset') will indicate to
        the tester to end the run and return the robot to the start.
        '''

        rotation = 0
        movement = 0
        
        # for report purposes
        old_location = [self.location[0], self.location[1]]
        
        # map surrounding
        self.map.expand(self.location, self.heading, sensors)
        
        # set visited
        self.map.visited[self.location[0]][self.location[1]] = 1
        
        # check if we have a 70% coverage of the map or time is elapsed
        if self.map.check_coverage() or self.timestep == self.max_timestep - self.timespan:
            # Debug Reports
            print (np.rot90(self.map.moved))
            
            # Reset everything
            self.reset()
            self.plan.reset()
            
            # different search path algorithms for best path
#             self.plan.path, self.plan.policy = self.plan.astar(True)
#             self.plan.path, self.plan.policy = self.plan.dp(True)
            self.plan.path, self.plan.policy = self.plan.recursive(True)
            
            # Reset map
            self.map.reset()
    
            # Reset run
            return ('Reset', 'Reset')    
        
        # check for exploration mode
        if self.plan.is_exploring:
            # check for goal
            self.plan.set_goal_reached()
            
            if self.plan.is_goal_reached:
                print('######################## GOAL REACHED #########################')
                # explore back to start in case it is not already doing so
                if not self.plan.is_looking_for_start:
                    self.plan.set_goal('Start')
                    self.plan.is_looking_for_start = True
                else:
                    self.plan.set_goal()
                    self.plan.is_looking_for_start = False
            
            # different explore algorithms
#             steering, movement = self.plan.random()
#             steering, movement = self.plan.astar()
            steering, movement = self.plan.recursive()
            print (np.rot90(self.map.moved))
            print (np.rot90(self.plan.policy))
            print (self.plan.path) 
    
        else:
            # walk by given path
            steering, movement = self.plan.set_steering_by_path(self.timestep), 1
            
            self.overall_timestep = self.timestep + self.explored_timestep
            print (self.overall_timestep, self.max_timestep)
            if self.overall_timestep == self.max_timestep:
                print (np.rot90(self.map.moved)) 
        
        
        # perform rotation and set new heading
        rotation = _dir_steering_to_rotation[steering]
        self.set_heading(steering)
        
        # perform movement
        if abs(movement) > 3:
            print "Movement limited to three squares in a turn."
        movement_steps = max(min(int(movement), 3), -3) # fix to range [-3, 3]
        while movement_steps:
            if movement_steps > 0:
                if movement_steps <= sensors[_dir_steering_to_sensor_index[steering]] and self.map.is_permissible(self.location, self.heading):
                    # map movement
                    self.map.moved[self.location[0]][self.location[1]] = _dir_heading_to_symbol[self.heading]

                    # perform move
                    self.location[0] += _dir_move[self.heading][0]
                    self.location[1] += _dir_move[self.heading][1]

                    self.map.moved[self.location[0]][self.location[1]] = "*"
                    movement_steps -= 1
                else:
                    print "Movement stopped by wall."
                    movement_steps = 0
            else:
                rev_heading = _dir_reverse[self.heading]
                if self.map.is_permissible(self.location, rev_heading):
                    # map movement
                    self.map.moved[self.location[0]][self.location[1]] = _dir_heading_to_symbol[rev_heading]

                    # perform move
                    self.location[0] += _dir_move[rev_heading][0]
                    self.location[1] += _dir_move[rev_heading][1]

                    self.map.moved[self.location[0]][self.location[1]] = "*"
                    movement_steps += 1
                else:
                    print "Movement stopped by wall."
                    movement_steps = 0
               
        # Debug Report
        print '{} [{:>2d},{:>2d},{:>2d}] {:>3d} => {}, {:>2d} steps = {}'.format(
            old_location,
            sensors[0], sensors[1], sensors[2],
            rotation,
            self.heading,
            movement,
            self.location)
        
        # increase time
        self.timestep += 1
        
        # return actions
        return rotation, movement

In [77]:
dir_sensors = {'u': ['l', 'u', 'r'], 'r': ['u', 'r', 'd'],
               'd': ['r', 'd', 'l'], 'l': ['d', 'l', 'u'],
               'up': ['l', 'u', 'r'], 'right': ['u', 'r', 'd'],
               'down': ['r', 'd', 'l'], 'left': ['d', 'l', 'u']}
dir_move = {'u': [0, 1], 'r': [1, 0], 'd': [0, -1], 'l': [-1, 0],
            'up': [0, 1], 'right': [1, 0], 'down': [0, -1], 'left': [-1, 0]}
dir_reverse = {'u': 'd', 'r': 'l', 'd': 'u', 'l': 'r',
               'up': 'd', 'right': 'l', 'down': 'u', 'left': 'r'}

# test and score parameters
max_time = _maxtimespan
train_score_mult = 1/30.

if __name__ == '__main__':
    '''
    This script tests a robot based on the code in robot.py on a maze given
    as an argument when running the script.
    '''

    # Create a maze based on input argument on command line.
    testmaze = Maze( str(dir_mazes[_maze]) )

    # Intitialize a robot; robot receives info about maze dimensions.
    testrobot = Robot(testmaze.dim)

    # Record robot performance over two runs.
    runtimes = []
    total_time = 0
    for run in range(2):
        print "Starting run {}.".format(run)

        # Set the robot in the start position. Note that robot position
        # parameters are independent of the robot itself.
        robot_pos = {'location': [0, 0], 'heading': 'up'}

        run_active = True
        hit_goal = False
        while run_active:
            # check for end of time
            total_time += 1
            if total_time > max_time:
                run_active = False
                print "Allotted time exceeded."
                break

            # provide robot with sensor information, get actions
            sensing = [testmaze.dist_to_wall(robot_pos['location'], heading)
                       for heading in dir_sensors[robot_pos['heading']]]
            
            rotation, movement = testrobot.next_move(sensing)

            # check for a reset
            if (rotation, movement) == ('Reset', 'Reset'):
                if run == 0 and hit_goal:
                    run_active = False
                    runtimes.append(total_time)
                    print "Ending first run. Starting next run."
                    break
                elif run == 0 and not hit_goal:
                    print "Cannot reset - robot has not hit goal yet."
                    continue
                else:
                    print "Cannot reset on runs after the first."
                    continue

            # perform rotation
            if rotation == -90:
                robot_pos['heading'] = dir_sensors[robot_pos['heading']][0]
            elif rotation == 90:
                robot_pos['heading'] = dir_sensors[robot_pos['heading']][2]
            elif rotation == 0:
                pass
            else:
                print "Invalid rotation value, no rotation performed."

            # perform movement
            if abs(movement) > 3:
                print "Movement limited to three squares in a turn."
            movement = max(min(int(movement), 3), -3) # fix to range [-3, 3]
            while movement:
                if movement > 0:
                    if testmaze.is_permissible(robot_pos['location'], robot_pos['heading']):
                        robot_pos['location'][0] += dir_move[robot_pos['heading']][0]
                        robot_pos['location'][1] += dir_move[robot_pos['heading']][1]
                        movement -= 1
                    else:
                        print "Movement stopped by wall."
                        movement = 0
                else:
                    rev_heading = dir_reverse[robot_pos['heading']]
                    if testmaze.is_permissible(robot_pos['location'], rev_heading):
                        robot_pos['location'][0] += dir_move[rev_heading][0]
                        robot_pos['location'][1] += dir_move[rev_heading][1]
                        movement += 1
                    else:
                        print "Movement stopped by wall."
                        movement = 0

            # check for goal enteqred
            goal_bounds = [testmaze.dim/2 - 1, testmaze.dim/2]
            if robot_pos['location'][0] in goal_bounds and robot_pos['location'][1] in goal_bounds:
                hit_goal = True
                if run != 0:
                    runtimes.append(total_time - sum(runtimes))
                    run_active = False
                    print "Goal found; run {} completed!".format(run)
                    
    # Report score if robot is successful.
    if len(runtimes) == 2:
        print "Task complete! Score: {:4.3f}".format(runtimes[1] + train_score_mult*runtimes[0])

Starting run 0.
[[' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ']]
[[' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' 'x' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' 